结构：

残差块——由残差块组成的模块——ResNet

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 残差块

残差块里首先有2个有相同输出通道数的3×3卷积层。

每个卷积层后接一个批量归一化和ReLU激活函数。

然后将输入跳过这两个卷积层直接加到ReLU激活函数之前。

这个设计**要求两个卷积层的输入与输入的形状一样**，从而可以相加。为此需要改变通道，我们引入一个额外的1x1卷积层来变换**输入**的形状。


In [20]:
class Residual(nn.Module):  # 本类已保存在d2lzh_pytorch包中方便以后使用
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1,stride=stride)
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1)
        if use_1x1conv:
            # 1x1卷积层，经过该层的数据，只会改变channel数。宽高不变。
            self.conv3 = nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self,X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X) # 1x1卷积层用于改变输入的形状
        return F.relu(Y + X)

检查ResModule的输入和输出**是否形状一致**。

In [10]:
blk = Residual(3,3)
X = torch.rand((4,3,6,6)) 
# print(X)
print(X.shape) # 输入
print(blk(X).shape) #输出

torch.Size([4, 3, 6, 6])
torch.Size([4, 3, 6, 6])


增加输入通道的同时，减半输出的高和宽

In [13]:
blk = Residual(3,6,use_1x1conv=True,stride=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

 # ResNet模型
ResNet则使用4个由残差块组成的模块block，每个模块使用若干个同样输出通道数的残差块。

第一个模块的通道数同输入通道数一致。

由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。

之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

In [21]:
# 每个block使用若干个残差块
def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    if first_block:
        assert in_channels == out_channels # 第一个模块的通道数同输入通道数一致
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

构建ResNet模型：

In [22]:
net = nn.Sequential(
    nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),
    nn.BatchNorm2d(64), # 64表示的是通道数
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3,stride=2, padding=1)
)
#接着我们为ResNet加入所有残差块。这里每个模块使用两个残差块。
net.add_module("resnet_block1", resnet_block(64, 64, 2, first_block=True))
net.add_module("resnet_block2", resnet_block(64, 128, 2))
net.add_module("resnet_block3", resnet_block(128, 256, 2))
net.add_module("resnet_block4", resnet_block(256, 512, 2))

与GoogleNet一样，加入平均池化层后接上全连接层输出：

In [23]:
net.add_module("global_avg_pool", d2l.GlobalAvgPool2d()) # GlobalAvgPool2d的输出: (Batch, 512, 1, 1)
net.add_module("fc", nn.Sequential(d2l.FlattenLayer(), nn.Linear(512, 10))) 

观察输入形状在ResNet不同模块之间的变化

In [24]:
X = torch.rand((1, 1, 224, 224))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)


0  output shape:	 torch.Size([1, 64, 112, 112])
1  output shape:	 torch.Size([1, 64, 112, 112])
2  output shape:	 torch.Size([1, 64, 112, 112])
3  output shape:	 torch.Size([1, 64, 56, 56])
resnet_block1  output shape:	 torch.Size([1, 64, 56, 56])
resnet_block2  output shape:	 torch.Size([1, 128, 28, 28])
resnet_block3  output shape:	 torch.Size([1, 256, 14, 14])
resnet_block4  output shape:	 torch.Size([1, 512, 7, 7])
global_avg_pool  output shape:	 torch.Size([1, 512, 1, 1])
fc  output shape:	 torch.Size([1, 10])
